

## Import libraries

In [1]:
import os
import ee
import geemap
import datetime
import pandas as pd

In [ ]:
# geemap.update_package()

## explore geemap 

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# Import the points where NO2 values will be pulled from GEE
in_csv = 'https://raw.githubusercontent.com/liyangyang515/NO2-in-South-East-Asia-_GE5219/main/data/Points_locations/SEA_pts.csv'

In [4]:
pts = geemap.csv_to_pandas(in_csv)
pts

,lon,lat
0,113.375,-8.625
1,113.625,-8.625
2,110.875,-8.375
3,111.125,-8.375
4,111.375,-8.375
...,...,...
8141,125.375,8.875
8142,125.625,8.875
8143,125.875,8.875
8144,126.125,8.875


In [6]:
# convert to ee points
ee_SEA = geemap.pandas_to_ee(pts, latitude="lat", longitude="lon")

In [7]:
# add points layer
Map.addLayer(ee_SEA, {}, 'SEA')

## create a loop

In [8]:
# get daily mean from Jun 8, 2020 to Dec 31, 2021
InitialDate= datetime.datetime(2020,6,8)
for i in range(0,572,1):
# for i in range(0,1,1):
    StartDate = (InitialDate + datetime.timedelta(days=i)).strftime('%Y-%m-%d')
    EndDate = (InitialDate + datetime.timedelta(days=(i+1))).strftime('%Y-%m-%d')
    collection = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2') \
    .select('tropospheric_NO2_column_number_density') \
    .filterDate(StartDate, EndDate) 
    # Reduce the collection with a mean reducer.
    mean = collection.reduce(ee.Reducer.mean())
    out_csv = r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily\NO2_' + StartDate + '.csv'
    geemap.extract_values_to_points(ee_SEA,mean,out_csv, scale = 27750)


Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily\NO2_2020-06-08.csv
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily\NO2_2020-06-09.csv
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily\NO2_2020-06-10.csv
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily\NO2_2020-06-11.csv
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily\NO2_2020-06-12.csv
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily\NO2_2020-06-13.csv
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_dai